In [59]:
import pandas as pd

In [60]:
commission_rate = 0.05

In [61]:
df = pd.read_excel('wagers.xlsx')

# Display the first few rowss to verify
print("Data Preview:")
df

Data Preview:


,Player,Side,Wager
0,Alice,Heads,324
1,Bob,Tails,832
2,Mark,Heads,432
3,Homer,Tails,516


In [62]:
sides = df['Side'].unique()
sides

array(['Heads', 'Tails'], dtype=object)

In [63]:
totals = df.groupby('Side')['Wager'].sum()
total_A = totals.get(sides[0], 0)
total_B = totals.get(sides[1], 0)

print(f"\nTotal wager on {sides[0]}: {total_A}")
print(f"Total wager on {sides[1]}: {total_B}")


Total wager on Heads: 756
Total wager on Tails: 1348


In [64]:
def calculate_payout(row, sides, total_A, total_B, commission_rate=0.05):
    """
    Calculate the payout for a player, taking into account the commission fee.

    :param row: The row containing the player's details (including 'Side' and 'Wager').
    :param sides: List of two sides ('Side A' and 'Side B').
    :param total_A: Total amount wagered on Side A.
    :param total_B: Total amount wagered on Side B.
    :param commission_rate: The commission fee percentage (default is 5%).
    :return: The net payout after commission.
    """
    side = row['Side']
    wager = row['Wager']
    
    # Calculate the payout based on the side that wins
    if side == sides[0]:  # Player bet on Side A
        payout = round(wager / total_A * total_B, 2) if total_A > 0 else 0
    elif side == sides[1]:  # Player bet on Side B
        payout = round(wager / total_B * total_A, 2) if total_B > 0 else 0
    else:
        return 0
    
    # Apply the commission fee
    commission = payout * commission_rate
    net_payout = round(payout - commission, 2)
    
    return net_payout


In [65]:
def calculate_gross_payout(row, sides, total_A, total_B, commission_rate=0.05):
    side = row['Side']
    wager = row['Wager']
    
    # Calculate the payout based on the side that wins
    if side == sides[0]:  # Player bet on Side A
        payout = round(wager / total_A * total_B, 2) if total_A > 0 else 0
    elif side == sides[1]:  # Player bet on Side B
        payout = round(wager / total_B * total_A, 2) if total_B > 0 else 0
    else:
        return 0
    
    # Apply the commission fee
    commission = payout * commission_rate
    net_payout = round(payout - commission, 2)
    
    return net_payout + row['Wager']

In [66]:
df[f'Payout_if_{sides[0]}_wins'] = df.apply(lambda row: calculate_payout(row, sides, total_A, total_B, commission_rate) if row['Side'] == sides[0] else 0, axis=1)
df[f'Payout_if_{sides[1]}_wins'] = df.apply(lambda row: calculate_payout(row, sides, total_A, total_B, commission_rate) if row['Side'] == sides[1] else 0, axis=1)

In [67]:
df[f'Total_Payout_if_{sides[0]}_wins'] = df.apply(lambda row: calculate_gross_payout(row, sides, total_A, total_B, commission_rate) if row['Side'] == sides[0] else 0, axis=1)
df[f'Total_Payout_if_{sides[1]}_wins'] = df.apply(lambda row: calculate_gross_payout(row, sides, total_A, total_B, commission_rate) if row['Side'] == sides[1] else 0, axis=1)

In [68]:
df

,Player,Side,Wager,Payout_if_Heads_wins,Payout_if_Tails_wins,Total_Payout_if_Heads_wins,Total_Payout_if_Tails_wins
0,Alice,Heads,324,548.82,0.00,872.82,0.00
1,Bob,Tails,832,0.00,443.28,0.00,1275.28
2,Mark,Heads,432,731.78,0.00,1163.78,0.00
3,Homer,Tails,516,0.00,274.92,0.00,790.92
